# Perceptual Loss trong Computer Vision

## Định nghĩa
**Perceptual Loss** (Loss tri giác) là một loại loss function được thiết kế để đo lường sự khác biệt giữa các ảnh dựa trên cách con người nhận thức thị giác, thay vì chỉ so sánh pixel theo pixel như L1 hoặc L2 loss.

## Tại sao cần Perceptual Loss?

### Vấn đề với Pixel-wise Loss:
- **L1/L2 Loss**: Chỉ so sánh từng pixel một cách độc lập
- **Kết quả**: Ảnh có thể có PSNR cao nhưng trông "mờ" hoặc thiếu chi tiết
- **Không phản ánh**: Cách con người đánh giá chất lượng ảnh

### Ưu điểm của Perceptual Loss:
- **Bảo toàn cấu trúc**: Giữ được các đặc trưng quan trọng của ảnh
- **Chất lượng thị giác**: Tạo ra ảnh sắc nét, chi tiết hơn
- **Phù hợp với nhận thức**: Gần với cách con người đánh giá ảnh

## Công thức toán học

### Pixel-wise Loss (L2):
```
L_pixel = ||I_pred - I_target||²
```

### Perceptual Loss:
```
L_perceptual = ||φ(I_pred) - φ(I_target)||²
```

Trong đó:
- `φ(·)`: Feature extractor (thường là CNN pre-trained như VGG)
- `I_pred`: Ảnh được tạo ra
- `I_target`: Ảnh ground truth

### Công thức chi tiết:
```
L_perceptual = Σ λᵢ * ||φᵢ(I_pred) - φᵢ(I_target)||²
```

Trong đó:
- `φᵢ`: Features từ layer thứ i
- `λᵢ`: Trọng số cho layer thứ i

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

class PerceptualLoss(nn.Module):
    def __init__(self, layers=['relu1_1', 'relu2_1', 'relu3_1', 'relu4_1']):
        super(PerceptualLoss, self).__init__()
        
        # Sử dụng VGG16 pre-trained
        vgg = models.vgg16(pretrained=True).features
        
        # Định nghĩa các layers cần extract features
        self.layer_names = layers
        self.layers = {}
        
        # Mapping layer names to indices in VGG
        layer_mapping = {
            'relu1_1': 1,   # after first ReLU
            'relu2_1': 6,   # after first ReLU in block 2
            'relu3_1': 11,  # after first ReLU in block 3
            'relu4_1': 18,  # after first ReLU in block 4
            'relu5_1': 25   # after first ReLU in block 5
        }
        
        # Extract specific layers
        for name in self.layer_names:
            if name in layer_mapping:
                layer_idx = layer_mapping[name]
                self.layers[name] = nn.Sequential(*list(vgg.children())[:layer_idx+1])
        
        # Freeze parameters
        for layer in self.layers.values():
            for param in layer.parameters():
                param.requires_grad = False
    
    def forward(self, pred, target):
        """
        Tính Perceptual Loss giữa predicted và target images
        
        Args:
            pred: Predicted image [B, 3, H, W]
            target: Target image [B, 3, H, W]
            
        Returns:
            perceptual_loss: Scalar loss value
        """
        total_loss = 0.0
        
        for layer_name, layer in self.layers.items():
            # Extract features
            pred_features = layer(pred)
            target_features = layer(target)
            
            # Compute L2 loss in feature space
            loss = F.mse_loss(pred_features, target_features)
            total_loss += loss
            
        return total_loss / len(self.layers)

# Example usage
perceptual_loss_fn = PerceptualLoss()

# Giả sử có 2 ảnh
batch_size = 4
channels = 3
height, width = 256, 256

pred_images = torch.randn(batch_size, channels, height, width)
target_images = torch.randn(batch_size, channels, height, width)

# Tính loss
loss = perceptual_loss_fn(pred_images, target_images)
print(f"Perceptual Loss: {loss.item():.4f}")

## So sánh các loại Loss

| Loss Type | Ưu điểm | Nhược điểm | Ứng dụng |
|-----------|---------|------------|----------|
| **L1/L2 Loss** | - Đơn giản<br>- Tính toán nhanh | - Ảnh mờ<br>- Mất chi tiết | Basic reconstruction |
| **Perceptual Loss** | - Chất lượng cao<br>- Bảo toàn cấu trúc | - Chậm hơn<br>- Cần pre-trained model | Style transfer, Super-resolution |
| **Adversarial Loss** | - Ảnh sắc nét<br>- Realistic | - Khó train<br>- Unstable | GAN-based generation |

## Ứng dụng trong Latent Diffusion Models

Trong paper "High-Resolution Image Synthesis with Latent Diffusion Models":

1. **VAE Training**: Sử dụng perceptual loss để train autoencoder
   ```python
   total_loss = reconstruction_loss + kl_loss + λ_perceptual * perceptual_loss
   ```

2. **Mục đích**: Đảm bảo VAE encode/decode giữ được thông tin thị giác quan trọng

3. **Kết quả**: Latent space có chất lượng cao hơn cho diffusion process

In [ ]:
# Ví dụ: VAE với Perceptual Loss (simplified)
class VAEWithPerceptualLoss(nn.Module):
    def __init__(self, encoder, decoder, latent_dim):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.perceptual_loss_fn = PerceptualLoss()
        
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        # Encode
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        
        # Decode
        x_recon = self.decoder(z)
        
        return x_recon, mu, logvar
    
    def loss_function(self, x, x_recon, mu, logvar, λ_perceptual=1.0, λ_kl=1.0):
        """
        Combined loss for VAE with perceptual loss
        """
        # Reconstruction loss (L2)
        recon_loss = F.mse_loss(x_recon, x, reduction='mean')
        
        # Perceptual loss
        perceptual_loss = self.perceptual_loss_fn(x_recon, x)
        
        # KL divergence
        kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
        
        # Total loss
        total_loss = recon_loss + λ_perceptual * perceptual_loss + λ_kl * kl_loss
        
        return {
            'total_loss': total_loss,
            'recon_loss': recon_loss,
            'perceptual_loss': perceptual_loss,
            'kl_loss': kl_loss
        }

print("VAE with Perceptual Loss implementation ready!")

## Tổng kết

### Perceptual Loss là gì?
- **Định nghĩa**: Loss function đo lường sự khác biệt dựa trên features thị giác
- **Cách hoạt động**: Sử dụng CNN pre-trained để extract features
- **Ưu điểm**: Tạo ra ảnh chất lượng cao, sắc nét hơn

### Vai trò trong Stable Diffusion:
1. **Training VAE**: Đảm bảo latent space có chất lượng cao
2. **Perceptual Compression**: Nén ảnh mà vẫn giữ được thông tin quan trọng
3. **Quality Control**: Kiểm soát chất lượng ảnh trong quá trình training

### References:
- [Perceptual Losses for Real-Time Style Transfer and Super-Resolution](https://arxiv.org/abs/1603.08155)
- [High-Resolution Image Synthesis with Latent Diffusion Models](https://arxiv.org/abs/2112.10752)
- [Photo-Realistic Single Image Super-Resolution Using a Generative Adversarial Network](https://arxiv.org/abs/1609.04802)

# Downsampling trong Computer Vision

## Định nghĩa
**Downsampling** (Lấy mẫu xuống) là quá trình **giảm kích thước hoặc độ phân giải** của dữ liệu bằng cách loại bỏ một số thông tin.

## Các loại Downsampling:

### 1. **Spatial Downsampling** (Giảm kích thước không gian):
- **Mục đích**: Giảm chiều rộng và chiều cao của ảnh
- **Ví dụ**: Ảnh 512x512 → 256x256
- **Phương pháp**:
  - Max Pooling
  - Average Pooling
  - Strided Convolution
  - Bilinear/Bicubic Interpolation

### 2. **Temporal Downsampling** (Giảm tần số thời gian):
- **Mục đích**: Giảm số frame trong video
- **Ví dụ**: 60fps → 30fps

### 3. **Channel Downsampling** (Giảm số kênh):
- **Mục đích**: Giảm chiều sâu của feature maps
- **Ví dụ**: 512 channels → 256 channels

## Công thức toán học

### Max Pooling:
```
Output[i,j] = max(Input[i*s:(i+1)*s, j*s:(j+1)*s])
```

### Average Pooling:
```
Output[i,j] = mean(Input[i*s:(i+1)*s, j*s:(j+1)*s])
```

### Strided Convolution:
```
Output = Conv2D(Input, kernel, stride=s)
```

Trong đó:
- `s`: Stride (bước nhảy)
- Kích thước output = ⌊(input_size - kernel_size) / stride⌋ + 1

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

# Ví dụ các phương pháp Downsampling
class DownsamplingMethods(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 1. Max Pooling
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # 2. Average Pooling
        self.avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)
        
        # 3. Strided Convolution
        self.strided_conv = nn.Conv2d(3, 3, kernel_size=3, stride=2, padding=1)
        
        # 4. Adaptive Average Pooling (cho kích thước cố định)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((128, 128))
    
    def forward(self, x):
        print(f"Input shape: {x.shape}")
        
        # Max pooling downsampling
        max_pooled = self.max_pool(x)
        print(f"Max pooled shape: {max_pooled.shape}")
        
        # Average pooling downsampling
        avg_pooled = self.avg_pool(x)
        print(f"Average pooled shape: {avg_pooled.shape}")
        
        # Strided convolution downsampling
        strided = self.strided_conv(x)
        print(f"Strided conv shape: {strided.shape}")
        
        # Adaptive pooling to fixed size
        adaptive = self.adaptive_pool(x)
        print(f"Adaptive pooled shape: {adaptive.shape}")
        
        return {
            'max_pooled': max_pooled,
            'avg_pooled': avg_pooled,
            'strided': strided,
            'adaptive': adaptive
        }

# Demo
downsampler = DownsamplingMethods()

# Tạo ảnh giả (batch_size=1, channels=3, height=256, width=256)
input_tensor = torch.randn(1, 3, 256, 256)
results = downsampler(input_tensor)

print("\n=== Downsampling Methods Demo ===")

In [ ]:
# Hàm downsampling thực tế
def downsample_image(image_tensor, factor=2, method='bilinear'):
    """
    Downsample ảnh với các phương pháp khác nhau
    
    Args:
        image_tensor: Tensor ảnh [B, C, H, W]
        factor: Hệ số giảm (2 = giảm một nửa)
        method: 'bilinear', 'nearest', 'area'
    
    Returns:
        Downsampled tensor
    """
    B, C, H, W = image_tensor.shape
    new_H, new_W = H // factor, W // factor
    
    return F.interpolate(
        image_tensor, 
        size=(new_H, new_W), 
        mode=method, 
        align_corners=False if method == 'bilinear' else None
    )

# Test downsampling function
original = torch.randn(1, 3, 512, 512)
print(f"Original size: {original.shape}")

# Downsample by factor of 2
downsampled_2x = downsample_image(original, factor=2)
print(f"Downsampled 2x: {downsampled_2x.shape}")

# Downsample by factor of 4
downsampled_4x = downsample_image(original, factor=4)
print(f"Downsampled 4x: {downsampled_4x.shape}")

# Downsample by factor of 8
downsampled_8x = downsample_image(original, factor=8)
print(f"Downsampled 8x: {downsampled_8x.shape}")

## So sánh các phương pháp Downsampling

| Phương pháp | Ưu điểm | Nhược điểm | Ứng dụng |
|-------------|---------|------------|----------|
| **Max Pooling** | - Bảo toàn đặc trưng quan trọng<br>- Invariant to small translations | - Mất thông tin<br>- Không smooth | CNN feature extraction |
| **Average Pooling** | - Smooth hơn<br>- Giảm noise | - Làm mờ edges<br>- Mất chi tiết | General downsampling |
| **Strided Convolution** | - Learnable<br>- Flexible | - Cần training<br>- More parameters | Modern CNN architectures |
| **Bilinear Interpolation** | - Smooth<br>- Continuous | - Computational cost<br>- Blurring | Image resizing |

## Vai trò trong Latent Diffusion Models

### 1. **VAE Encoder Downsampling**:
```python
# Trong VAE encoder
x = downsample_block(x)  # 512x512 → 256x256
x = downsample_block(x)  # 256x256 → 128x128  
x = downsample_block(x)  # 128x128 → 64x64
# Kết quả: latent space 64x64 thay vì 512x512
```

### 2. **Computational Efficiency**:
- **Giảm memory**: 512² = 262,144 pixels → 64² = 4,096 pixels (64x ít hơn)
- **Tăng tốc**: Diffusion process chạy trên latent space nhỏ hơn
- **Scalability**: Có thể xử lý ảnh độ phân giải cao

### 3. **Multi-scale Processing**:
- U-Net sử dụng nhiều mức downsampling
- Skip connections để bảo toàn thông tin
- Progressive refinement

In [ ]:
# Ví dụ: VAE Encoder với Downsampling (simplified)
class VAEEncoderWithDownsampling(nn.Module):
    def __init__(self, input_channels=3, latent_dim=512):
        super().__init__()
        
        # Progressive downsampling
        self.encoder = nn.Sequential(
            # 512x512 → 256x256
            nn.Conv2d(input_channels, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            
            # 256x256 → 128x128
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.ReLU(),
            
            # 128x128 → 64x64
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.ReLU(),
            
            # 64x64 → 32x32
            nn.Conv2d(256, 512, 4, stride=2, padding=1),
            nn.ReLU(),
            
            # 32x32 → 16x16
            nn.Conv2d(512, 512, 4, stride=2, padding=1),
            nn.ReLU(),
        )
        
        # Final layers cho mu và logvar
        self.fc_mu = nn.Conv2d(512, latent_dim, 1)
        self.fc_logvar = nn.Conv2d(512, latent_dim, 1)
    
    def forward(self, x):
        print(f"Input: {x.shape}")
        
        # Progressive downsampling
        features = self.encoder(x)
        print(f"After downsampling: {features.shape}")
        
        # Generate mu and logvar
        mu = self.fc_mu(features)
        logvar = self.fc_logvar(features)
        
        print(f"Latent mu: {mu.shape}")
        print(f"Latent logvar: {logvar.shape}")
        
        return mu, logvar

# Demo VAE Encoder
encoder = VAEEncoderWithDownsampling()
input_image = torch.randn(1, 3, 512, 512)
mu, logvar = encoder(input_image)

print(f"\nDownsampling ratio: {512//16}x (512x512 → 16x16)")
print(f"Memory reduction: {(512*512)/(16*16):.1f}x")

## Tổng kết về Downsampling

### Downsampling là gì?
- **Định nghĩa**: Quá trình giảm kích thước hoặc độ phân giải của dữ liệu
- **Mục đích**: Giảm computational cost, memory usage, và tăng receptive field
- **Trade-off**: Giảm chi tiết nhưng tăng efficiency

### Các phương pháp chính:
1. **Max/Average Pooling**: Đơn giản, nhanh
2. **Strided Convolution**: Learnable, linh hoạt  
3. **Interpolation**: Smooth, continuous

### Vai trò trong Stable Diffusion:
1. **VAE Compression**: Giảm ảnh 512x512 → latent 64x64
2. **Efficiency**: Diffusion process chạy nhanh hơn 64x
3. **Scalability**: Xử lý được ảnh high-resolution
4. **Quality**: Vẫn bảo toàn thông tin quan trọng nhờ perceptual loss

### Key Benefits:
- **Memory**: Giảm 64x memory usage
- **Speed**: Tăng 64x training/inference speed  
- **Quality**: Maintained through perceptual compression
- **Flexibility**: Support nhiều resolutions

# High Variance trong Machine Learning

## Định nghĩa
**High Variance** (Phương sai cao) là một hiện tượng trong machine learning khi model **quá nhạy cảm** với những thay đổi nhỏ trong training data, dẫn đến kết quả **không ổn định** và **khó dự đoán**.

## Đặc điểm của High Variance:

### 1. **Overfitting**:
- Model học quá chi tiết từ training data
- Performance tốt trên training set nhưng kém trên validation/test set
- Model "ghi nhớ" noise thay vì học pattern thực sự

### 2. **Instability** (Không ổn định):
- Kết quả thay đổi lớn khi thay đổi training data một chút
- Model predictions không consistent
- High sensitivity to random fluctuations

### 3. **Poor Generalization**:
- Không generalize tốt cho unseen data
- Gap lớn giữa training và validation performance
- Model quá "specific" cho training examples

## Công thức Toán học

### Variance của Model:
```
Variance = E[(f(x) - E[f(x)])²]
```

### Bias-Variance Tradeoff:
```
Total Error = Bias² + Variance + Irreducible Error
```

Trong đó:
- **Bias**: Sai số systematic do model quá đơn giản
- **Variance**: Sai số do model quá phức tạp và unstable
- **Irreducible Error**: Noise inherent trong data

### High Variance Indicators:
- **Training Error << Validation Error**
- **Large gap between train/val performance**
- **Model predictions vary widely với small data changes**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Tạo synthetic data
np.random.seed(42)
n_samples = 100
X = np.linspace(0, 1, n_samples).reshape(-1, 1)
y = 1.5 * X.ravel() + 0.3 * np.sin(15 * X.ravel()) + 0.1 * np.random.randn(n_samples)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Demonstrate High Variance với Polynomial Regression
def demonstrate_variance(degrees, n_experiments=50):
    """
    Demonstrate high variance với polynomial regression
    """
    results = {}
    
    for degree in degrees:
        train_errors = []
        test_errors = []
        predictions = []
        
        # Multiple experiments với different random splits
        for i in range(n_experiments):
            # Random split mỗi lần
            X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3, random_state=i)
            
            # Create polynomial model
            poly_model = Pipeline([
                ('poly', PolynomialFeatures(degree=degree)),
                ('linear', LinearRegression())
            ])
            
            # Train model
            poly_model.fit(X_tr, y_tr)
            
            # Predictions
            y_train_pred = poly_model.predict(X_tr)
            y_test_pred = poly_model.predict(X_te)
            
            # Calculate errors
            train_error = mean_squared_error(y_tr, y_train_pred)
            test_error = mean_squared_error(y_te, y_test_pred)
            
            train_errors.append(train_error)
            test_errors.append(test_error)
            
            # Store predictions for visualization
            if i < 10:  # Chỉ store first 10 experiments
                X_plot = np.linspace(0, 1, 100).reshape(-1, 1)
                y_plot_pred = poly_model.predict(X_plot)
                predictions.append(y_plot_pred)
        
        results[degree] = {
            'train_errors': train_errors,
            'test_errors': test_errors, 
            'predictions': predictions,
            'train_mean': np.mean(train_errors),
            'train_std': np.std(train_errors),
            'test_mean': np.mean(test_errors),
            'test_std': np.std(test_errors)
        }
    
    return results

# Test với different polynomial degrees
degrees = [1, 3, 9, 15]  # Low to High complexity
results = demonstrate_variance(degrees)

# Print results
print("=== Bias-Variance Analysis ===")
print(f"{'Degree':<8} {'Train Mean':<12} {'Train Std':<12} {'Test Mean':<12} {'Test Std':<12} {'Variance':<10}")
print("-" * 70)

for degree in degrees:
    r = results[degree]
    variance_indicator = "HIGH" if r['test_std'] > 0.05 else "LOW"
    print(f"{degree:<8} {r['train_mean']:<12.4f} {r['train_std']:<12.4f} {r['test_mean']:<12.4f} {r['test_std']:<12.4f} {variance_indicator:<10}")

## So sánh High Bias vs High Variance

| Aspect | High Bias (Underfitting) | High Variance (Overfitting) |
|--------|---------------------------|------------------------------|
| **Training Error** | High | Low |
| **Validation Error** | High | High |
| **Error Gap** | Small | Large |
| **Model Complexity** | Too Simple | Too Complex |
| **Symptoms** | Poor performance everywhere | Good on train, bad on validation |
| **Example** | Linear model cho non-linear data | Deep network với ít data |

## Cách nhận biết High Variance:

### 1. **Performance Metrics**:
```python
# High Variance indicators
training_accuracy = 0.95
validation_accuracy = 0.65
gap = training_accuracy - validation_accuracy  # 0.30 (large gap!)

if gap > 0.15:  # Threshold example
    print("High Variance detected!")
```

### 2. **Learning Curves**:
- Training error giảm liên tục
- Validation error tăng hoặc plateau
- Gap lớn và persistent giữa train/val curves

### 3. **Cross-Validation**:
- High standard deviation across folds
- Inconsistent performance across different data splits

## Giải pháp cho High Variance

### 1. **Regularization**:
```python
# L1/L2 Regularization
from sklearn.linear_model import Ridge, Lasso

# L2 Regularization (Ridge)
ridge_model = Ridge(alpha=1.0)

# L1 Regularization (Lasso)
lasso_model = Lasso(alpha=0.1)
```

### 2. **More Training Data**:
- Collect more samples
- Data augmentation
- Synthetic data generation

### 3. **Reduce Model Complexity**:
```python
# Giảm parameters
- Fewer layers trong neural networks
- Lower polynomial degree
- Feature selection
- Pruning
```

### 4. **Ensemble Methods**:
```python
# Bagging reduces variance
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100)

# Voting classifier
from sklearn.ensemble import VotingClassifier
ensemble = VotingClassifier([('model1', model1), ('model2', model2)])
```

### 5. **Dropout và Early Stopping**:
```python
# For neural networks
import torch.nn as nn

class ModelWithDropout(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(100, 50)
        self.dropout = nn.Dropout(0.3)  # Giảm overfitting
        self.layer2 = nn.Linear(50, 1)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout(x)  # Randomly zero out neurons
        return self.layer2(x)
```

## High Variance trong Diffusion Models

### 1. **Sampling Variance**:
Trong diffusion models, sampling process có thể có high variance:

```python
# Multiple samples từ cùng một noise
for i in range(5):
    noise = torch.randn_like(latent)  # Same shape, different random values
    sample = diffusion_model.sample(noise, prompt)
    # Kết quả có thể vary significantly
```

### 2. **Training Instability**:
- Diffusion loss có thể fluctuate wildly
- Gradient variance cao do random timestep sampling
- Model weights update inconsistently

### 3. **Solutions trong Stable Diffusion**:

#### **Classifier-Free Guidance**:
```python
# Reduce variance bằng guidance
guided_prediction = unconditional_pred + guidance_scale * (conditional_pred - unconditional_pred)
# guidance_scale giúp control variance vs quality tradeoff
```

#### **Variance Reduction Techniques**:
```python
# 1. Antithetic sampling
noise_1 = torch.randn_like(x)
noise_2 = -noise_1  # Antithetic pair

# 2. Low-discrepancy sequences thay vì pure random
# 3. Importance sampling cho timesteps
```

#### **Progressive Training**:
- Start với simple tasks (low variance)
- Gradually increase complexity
- Curriculum learning approach

### 4. **VAE Regularization**:
```python
# KL divergence trong VAE giúp control variance
kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
# Beta-VAE: beta * kl_loss (beta > 1 reduces variance)
```

In [ ]:
# Practical Example: Detecting High Variance trong Training
class VarianceMonitor:
    def __init__(self, window_size=100):
        self.window_size = window_size
        self.train_losses = []
        self.val_losses = []
        self.predictions_history = []
    
    def update(self, train_loss, val_loss, predictions=None):
        self.train_losses.append(train_loss)
        self.val_losses.append(val_loss)
        if predictions is not None:
            self.predictions_history.append(predictions)
    
    def check_variance(self):
        if len(self.train_losses) < self.window_size:
            return "Insufficient data"
        
        recent_train = self.train_losses[-self.window_size:]
        recent_val = self.val_losses[-self.window_size:]
        
        # Check gap between train and validation
        avg_train = np.mean(recent_train)
        avg_val = np.mean(recent_val)
        gap = avg_val - avg_train
        
        # Check stability (variance of losses)
        train_variance = np.var(recent_train)
        val_variance = np.var(recent_val)
        
        # Check prediction consistency
        pred_variance = 0
        if len(self.predictions_history) >= 5:
            recent_preds = self.predictions_history[-5:]
            pred_variance = np.var([np.mean(pred) for pred in recent_preds])
        
        results = {
            'train_val_gap': gap,
            'train_variance': train_variance,
            'val_variance': val_variance,
            'prediction_variance': pred_variance,
            'high_variance_detected': gap > 0.1 or val_variance > 0.05
        }
        
        return results
    
    def suggest_solutions(self):
        analysis = self.check_variance()
        suggestions = []
        
        if analysis['high_variance_detected']:
            suggestions.append("🚨 High Variance Detected!")
            
            if analysis['train_val_gap'] > 0.1:
                suggestions.extend([
                    "• Add regularization (L1/L2, Dropout)",
                    "• Collect more training data", 
                    "• Reduce model complexity",
                    "• Use early stopping"
                ])
            
            if analysis['val_variance'] > 0.05:
                suggestions.extend([
                    "• Use ensemble methods",
                    "• Implement cross-validation",
                    "• Check data quality"
                ])
                
            if analysis['prediction_variance'] > 0.1:
                suggestions.extend([
                    "• Increase training epochs",
                    "• Adjust learning rate",
                    "• Use learning rate scheduling"
                ])
        else:
            suggestions.append("✅ Variance levels look healthy!")
        
        return suggestions

# Demo usage
monitor = VarianceMonitor()

# Simulate training với high variance
for epoch in range(200):
    # Simulate decreasing train loss but fluctuating val loss
    train_loss = 1.0 * np.exp(-epoch/50) + 0.01 * np.random.randn()
    val_loss = 0.5 + 0.3 * np.sin(epoch/10) + 0.1 * np.random.randn()
    
    monitor.update(train_loss, val_loss)
    
    if epoch % 50 == 0 and epoch > 100:
        analysis = monitor.check_variance()
        suggestions = monitor.suggest_solutions()
        
        print(f"\nEpoch {epoch} Analysis:")
        print(f"Train-Val Gap: {analysis['train_val_gap']:.3f}")
        print(f"Validation Variance: {analysis['val_variance']:.3f}")
        print("Suggestions:")
        for suggestion in suggestions:
            print(f"  {suggestion}")

## Tổng kết về High Variance

### High Variance là gì?
- **Định nghĩa**: Model quá nhạy cảm với changes trong training data
- **Triệu chứng**: Overfitting, performance gap lớn, predictions không stable
- **Nguyên nhân**: Model quá complex, data quá ít, lack of regularization

### Key Indicators:
1. **Large Train-Validation Gap**: Gap > 10-15%
2. **High Standard Deviation**: Trong cross-validation results 
3. **Unstable Predictions**: Vary widely với small data changes
4. **Learning Curves**: Train error giảm nhưng val error tăng

### Main Solutions:
1. **Regularization**: L1/L2, Dropout, Early Stopping
2. **More Data**: Collection, Augmentation, Synthesis
3. **Model Simplification**: Fewer parameters, Feature selection
4. **Ensemble Methods**: Bagging, Voting, Stacking
5. **Cross-Validation**: Better evaluation và model selection

### Trong Diffusion Models:
- **Sampling variance**: Multiple runs give different results
- **Training instability**: Loss fluctuations, gradient variance
- **Solutions**: Classifier-free guidance, antithetic sampling, progressive training

### Remember:
**High Variance = High Complexity + Low Stability**
- Trade-off với bias: Reducing variance might increase bias
- Goal: Find optimal balance for best generalization
- Monitor continuously during training process

### Key Takeaway:
*"A model with high variance is like a weather vane - it moves dramatically with small changes in the wind (data), making it unreliable for consistent predictions."*

# Diffusion Models - Hiểu sâu về cơ chế hoạt động

## Định nghĩa cơ bản
**Diffusion Models** là các **mô hình xác suất** được thiết kế để học phân phối dữ liệu `p(x)` bằng cách **từ từ khử nhiễu** một biến có phân phối chuẩn.

## Ý tưởng chính

### 1. **Quá trình ngược của Markov Chain**:
- Diffusion models học **quá trình ngược** của một chuỗi Markov có độ dài T
- **Forward process**: x₀ → x₁ → x₂ → ... → xₜ (thêm nhiễu dần)
- **Reverse process**: xₜ → xₜ₋₁ → ... → x₁ → x₀ (khử nhiễu dần)

### 2. **Từ nhiễu đến ảnh thật**:
```
Noise ~ N(0,1) → [Diffusion Model] → Real Image
```

## Cách hoạt động chi tiết

### **Forward Process (Thêm nhiễu)**:
```
q(x₁:ₜ|x₀) = ∏ q(xₜ|xₜ₋₁)
```
- Bắt đầu từ ảnh thật x₀
- Từ từ thêm nhiễu Gaussian ở mỗi bước
- Cuối cùng có nhiễu thuần túy xₜ ~ N(0,1)

### **Reverse Process (Khử nhiễu)**:
```
pθ(x₀:ₜ₋₁|xₜ) = ∏ pθ(xₜ₋₁|xₜ)
```
- Bắt đầu từ nhiễu xₜ
- Model học cách **đoán nhiễu** để loại bỏ
- Từ từ tạo ra ảnh thật x₀

## Công thức toán học quan trọng

### **Variational Lower Bound**:
Diffusion models sử dụng một biến thể của **variational lower bound** trên p(x):

```
log p(x) ≥ E[log pθ(x₀|x₁)] - KL[q(x₁|x₀)||pθ(x₁)] - ...
```

### **Denoising Score Matching**:
Phương pháp này tương đương với **denoising score-matching**:
- Thay vì học p(x) trực tiếp
- Model học **score function**: ∇ₓ log p(x)
- Qua việc dự đoán nhiễu cần loại bỏ

### **Simplified Loss Function**:
Loss function được đơn giản hóa thành:

```
LDM = Ex,ε~N(0,1),t [||ε - εθ(xt, t)||₂²]
```

**Giải thích**:
- `x`: Ảnh gốc (clean image)
- `ε ~ N(0,1)`: Nhiễu ngẫu nhiên được thêm vào
- `t`: Timestep được chọn ngẫu nhiên từ {1,...,T}
- `xt`: Ảnh đã bị nhiễu ở timestep t
- `εθ(xt, t)`: Model dự đoán nhiễu
- `||ε - εθ(xt, t)||₂²`: Sai số L2 giữa nhiễu thật và nhiễu dự đoán

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class SimpleDiffusionLoss(nn.Module):
    def __init__(self, num_timesteps=1000):
        super().__init__()
        self.num_timesteps = num_timesteps
        
        # Tạo noise schedule (beta values)
        self.betas = torch.linspace(0.0001, 0.02, num_timesteps)
        self.alphas = 1.0 - self.betas
        self.alphas_cumprod = torch.cumprod(self.alphas, dim=0)
    
    def add_noise(self, x0, noise, timesteps):
        """
        Thêm nhiễu vào ảnh gốc theo công thức:
        xt = sqrt(alphas_cumprod_t) * x0 + sqrt(1 - alphas_cumprod_t) * noise
        """
        sqrt_alphas_cumprod_t = torch.sqrt(self.alphas_cumprod[timesteps])
        sqrt_one_minus_alphas_cumprod_t = torch.sqrt(1.0 - self.alphas_cumprod[timesteps])
        
        # Reshape để broadcast đúng
        sqrt_alphas_cumprod_t = sqrt_alphas_cumprod_t.view(-1, 1, 1, 1)
        sqrt_one_minus_alphas_cumprod_t = sqrt_one_minus_alphas_cumprod_t.view(-1, 1, 1, 1)
        
        return sqrt_alphas_cumprod_t * x0 + sqrt_one_minus_alphas_cumprod_t * noise
    
    def forward(self, model, x0):
        """
        Tính diffusion loss
        
        Args:
            model: Neural network dự đoán nhiễu εθ(xt, t)
            x0: Batch ảnh gốc [B, C, H, W]
        
        Returns:
            loss: Scalar loss value
        """
        batch_size = x0.shape[0]
        
        # 1. Sample random noise ε ~ N(0,1)
        noise = torch.randn_like(x0)
        
        # 2. Sample random timesteps t
        timesteps = torch.randint(0, self.num_timesteps, (batch_size,), device=x0.device)
        
        # 3. Add noise to get xt
        xt = self.add_noise(x0, noise, timesteps)
        
        # 4. Model dự đoán nhiễu
        predicted_noise = model(xt, timesteps)
        
        # 5. Tính L2 loss giữa nhiễu thật và dự đoán
        loss = F.mse_loss(predicted_noise, noise)
        
        return loss

# Ví dụ sử dụng
class SimpleUNet(nn.Module):
    """Simplified U-Net cho demo"""
    def __init__(self, in_channels=3, time_emb_dim=128):
        super().__init__()
        self.time_mlp = nn.Sequential(
            nn.Linear(1, time_emb_dim),
            nn.ReLU(),
            nn.Linear(time_emb_dim, time_emb_dim)
        )
        
        # Simplified encoder-decoder
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
        )
        
        self.decoder = nn.Sequential(
            nn.Conv2d(64 + time_emb_dim, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, in_channels, 3, padding=1)
        )
    
    def forward(self, x, t):
        # Time embedding
        t_emb = self.time_mlp(t.float().unsqueeze(-1))  # [B, time_emb_dim]
        t_emb = t_emb.view(t_emb.shape[0], t_emb.shape[1], 1, 1)  # [B, time_emb_dim, 1, 1]
        t_emb = t_emb.expand(-1, -1, x.shape[2], x.shape[3])  # [B, time_emb_dim, H, W]
        
        # Encoder
        x_enc = self.encoder(x)
        
        # Combine with time embedding
        x_combined = torch.cat([x_enc, t_emb], dim=1)
        
        # Decoder (predict noise)
        noise_pred = self.decoder(x_combined)
        
        return noise_pred

# Demo
model = SimpleUNet()
loss_fn = SimpleDiffusionLoss(num_timesteps=1000)

# Tạo batch ảnh giả
batch_size = 4
images = torch.randn(batch_size, 3, 64, 64)  # [B, C, H, W]

# Tính loss
loss = loss_fn(model, images)
print(f"Diffusion Loss: {loss.item():.4f}")

# Giải thích quá trình:
print("\n=== Quá trình Training Diffusion Model ===")
print("1. Lấy ảnh gốc x0")
print("2. Sample nhiễu ε ~ N(0,1)")
print("3. Sample timestep t ngẫu nhiên")
print("4. Tạo ảnh nhiễu xt = √(ᾱt) * x0 + √(1-ᾱt) * ε")
print("5. Model dự đoán nhiễu: εθ(xt, t)")
print("6. Tính loss: ||ε - εθ(xt, t)||²")
print("7. Backprop và update weights")

## Denoising Autoencoders εθ(xt, t)

### **Ý tưởng chính**:
Diffusion models có thể được hiểu như một **chuỗi các denoising autoencoders** có trọng số bằng nhau:
- **εθ(xt, t)** với t = 1, 2, ..., T
- Mỗi autoencoder được train để dự đoán nhiễu trong ảnh xt
- **xt** là phiên bản nhiễu của ảnh đầu vào x

### **Tại sao gọi là "Equally weighted sequence"?**
```
LDM = Ex,ε~N(0,1),t [||ε - εθ(xt, t)||²]
```
- Mỗi timestep t có **trọng số bằng nhau** (equally weighted)
- Không có λt trong công thức (khác với original DDPM)
- Đây là **simplified version** của variational lower bound

### **Input và Output**:
- **Input**: 
  - `xt`: Ảnh đã bị nhiễu ở timestep t
  - `t`: Timestep (cho model biết mức độ nhiễu)
- **Output**: 
  - `εθ(xt, t)`: Dự đoán nhiễu cần loại bỏ

### **Denoised variant**:
- Model không dự đoán ảnh sạch x0 trực tiếp
- Mà dự đoán **nhiễu ε** để loại bỏ
- Từ đó tính ra ảnh sạch: `x0 ≈ (xt - √(1-ᾱt) * εθ(xt,t)) / √(ᾱt)`

## Giải thích đoạn văn trong paper

> *"Diffusion Models [82] are probabilistic models designed to learn a data distribution p(x) by gradually denoising a normally distributed variable, which corresponds to learning the reverse process of a fixed Markov Chain of length T."*

**Dịch và giải thích**:
- **"Probabilistic models"**: Mô hình xác suất
- **"Learn a data distribution p(x)"**: Học phân phối dữ liệu (ví dụ: phân phối của tất cả ảnh mèo)
- **"Gradually denoising"**: Từ từ khử nhiễu (không phải một lần)
- **"Normally distributed variable"**: Biến có phân phối chuẩn (Gaussian noise)
- **"Reverse process of fixed Markov Chain"**: Quá trình ngược của chuỗi Markov cố định

> *"For image synthesis, the most successful models [15,30,72] rely on a reweighted variant of the variational lower bound on p(x), which mirrors denoising score-matching [85]."*

**Giải thích**:
- **"Reweighted variant"**: Biến thể có trọng số khác của variational lower bound
- **"Mirrors denoising score-matching"**: Tương đương với phương pháp denoising score-matching
- Thay vì dùng công thức phức tạp, họ đơn giản hóa thành MSE loss

> *"These models can be interpreted as an equally weighted sequence of denoising autoencoders εθ(xt,t); t = 1...T, which are trained to predict a denoised variant of their input xt, where xt is a noisy version of the input x."*

**Giải thích**:
- **"Equally weighted sequence"**: Chuỗi có trọng số bằng nhau
- **"Denoising autoencoders"**: Các autoencoder khử nhiễu
- **"Predict a denoised variant"**: Dự đoán phiên bản đã khử nhiễu
- Thực tế: model dự đoán **nhiễu** chứ không phải ảnh sạch trực tiếp

> *"The corresponding objective can be simplified to: LDM = Ex,ε~N(0,1),t [||ε - εθ(xt,t)||²]"*

**Giải thích công thức**:
- **E**: Kỳ vọng (expected value)
- **x**: Ảnh từ dataset
- **ε ~ N(0,1)**: Nhiễu Gaussian
- **t**: Timestep uniform từ {1,...,T}
- **||ε - εθ(xt,t)||²**: L2 loss giữa nhiễu thật và dự đoán

### **Tóm lại**:
Đoạn văn giải thích rằng Diffusion Models:
1. **Học phân phối dữ liệu** bằng cách khử nhiễu từ từ
2. **Tương đương** với chuỗi denoising autoencoders
3. **Training đơn giản**: chỉ cần dự đoán nhiễu với MSE loss
4. **Hiệu quả**: thay thế công thức phức tạp bằng công thức đơn giản

Đây chính là **nền tảng** cho Latent Diffusion Models - áp dụng nguyên lý này trong latent space thay vì pixel space!

## Hiểu theo cách Việt Nam 🇻🇳

### **Ví dụ đơn giản**:
Tưởng tượng bạn đang **vẽ tranh**:

1. **Forward process** (thêm nhiễu):
   - Bắt đầu: Bức tranh đẹp 🎨
   - Bước 1: Rắc một ít bụi lên tranh 🌫️
   - Bước 2: Rắc thêm bụi 🌫️🌫️
   - ...
   - Cuối cùng: Chỉ còn toàn bụi trắng ⬜

2. **Reverse process** (khử nhiễu):
   - Bắt đầu: Tờ giấy toàn bụi trắng ⬜
   - Model học: "Nhìn tờ giấy này, tôi đoán cần lau đi những bụi nào?"
   - Từ từ lau sạch → Xuất hiện nét vẽ → Dần dần thành tranh đẹp 🎨

### **Tại sao gọi là "Equally weighted"?**
- Giống như **học từng cấp độ** trong trường học
- Lớp 1, lớp 2, ..., lớp 12 đều **quan trọng như nhau**
- Không phải lớp 12 quan trọng hơn lớp 1
- Diffusion model cũng vậy: mọi timestep đều có trọng số bằng nhau

### **Denoising autoencoders**:
- **Autoencoder**: Máy nén và giải nén
- **Denoising**: Chuyên khử nhiễu
- Giống như có **1000 thợ sửa tranh**, mỗi thợ chuyên sửa một mức độ hỏng khác nhau
- Thợ số 1: Sửa tranh hỏng ít
- Thợ số 1000: Sửa tranh hỏng nhiều (gần như toàn bụi)

### **Tại sao Diffusion thành công?**
1. **Chia để trị**: Thay vì tạo ảnh một lút → Chia thành 1000 bước nhỏ
2. **Ổn định**: Không bị "điên" như GAN
3. **Linh hoạt**: Có thể điều khiển bằng text
4. **Chất lượng cao**: Tạo ảnh realistic

### **Kết nối với Stable Diffusion**:
- **Stable Diffusion** = Diffusion Models + VAE + Text Conditioning
- Thay vì làm trên ảnh 512×512 → Làm trên latent 64×64 (nhanh hơn 64 lần!)
- Kết quả: Tạo ảnh chất lượng cao, nhanh, và có thể điều khiển bằng text

**🎯 Mục tiêu cuối cùng**: Từ câu text "một con mèo đang ngồi trên ghế" → Tạo ra ảnh mèo đẹp và đúng mô tả!

# Stable Diffusion Model Architecture & Training Pipeline 🏗️

## Tổng quan Architecture

**Stable Diffusion** không phải là một model đơn lẻ, mà là **hệ thống gồm 3 components chính**:

### 1. **First Stage Model (VAE)**:
- **Encoder**: E(x) → z (ảnh → latent)
- **Decoder**: D(z) → x (latent → ảnh)
- **Mục đích**: Nén ảnh từ 512×512 → latent 64×64 (giảm 64x)

### 2. **Diffusion Model (U-Net)**:
- **Input**: Noisy latent zt, timestep t, conditioning c
- **Output**: Predicted noise εθ(zt, t, c)
- **Mục đích**: Học khử nhiễu trong latent space

### 3. **Conditioning Encoder**:
- **Text Encoder**: CLIP hoặc T5 (text → embedding)
- **Cross-attention**: Inject text vào U-Net
- **Mục đích**: Điều khiển generation bằng text

## Kiến trúc tổng thể:
```
Text Prompt → [CLIP] → Text Embedding
                            ↓
Noise → [U-Net + Cross-Attention] → Clean Latent → [VAE Decoder] → Final Image
```

# 3 Giai đoạn Training của Stable Diffusion 🎯

## Giai đoạn 1: Pre-training VAE (Autoencoder)

### **Mục tiêu**: Tạo ra một VAE chất lượng cao để nén ảnh

### **Training Process**:
```python
# VAE Loss Function
total_loss = reconstruction_loss + β * kl_loss + λ * perceptual_loss + adversarial_loss
```

### **Components**:
1. **Reconstruction Loss**: L2 loss giữa input và reconstructed image
2. **KL Divergence**: Regularize latent space
3. **Perceptual Loss**: VGG-based features để bảo toàn visual quality
4. **Adversarial Loss**: GAN loss để tạo ảnh realistic

### **Dataset**: 
- LAION-400M (400 triệu ảnh-text pairs)
- ImageNet
- Other large-scale image datasets

### **Result**: 
- VAE có thể encode ảnh 512×512 → latent 64×64
- Decode latent → ảnh chất lượng cao
- Compression ratio: 8×8×3 = 192x (thực tế ~64x do latent channels)

---

## Giai đoạn 2: Training Diffusion Model trong Latent Space

### **Mục tiêu**: Học diffusion process trong latent space của VAE

### **Training Process**:
```python
# Latent Diffusion Loss
LLDM = Ez~E(x),ε~N(0,1),t [||ε - εθ(zt, t)||²]
```

### **Steps**:
1. **Encode images**: x → z = E(x) bằng pre-trained VAE
2. **Add noise**: zt = √(ᾱt) * z + √(1-ᾱt) * ε  
3. **Train U-Net**: Dự đoán noise εθ(zt, t)
4. **Backprop**: Minimize MSE loss

### **U-Net Architecture**:
- **Input**: Noisy latent zt [B, 4, 64, 64]
- **Time embedding**: Sinusoidal encoding của timestep t
- **Skip connections**: Encoder-decoder với residual connections
- **Attention**: Self-attention ở multiple resolutions

### **Training Details**:
- **Timesteps**: T = 1000
- **Noise schedule**: Linear hoặc cosine
- **Batch size**: Large (depends on hardware)
- **Learning rate**: 1e-4 với cosine annealing

---

## Giai đoạn 3: Adding Conditioning (Text-to-Image)

### **Mục tiêu**: Thêm khả năng điều khiển generation bằng text

### **Architecture Changes**:
```python
# Conditioned Diffusion Loss  
LLDM = Ez~E(x),c,ε~N(0,1),t [||ε - εθ(zt, t, c)||²]
```

### **Text Conditioning Process**:
1. **Text Encoding**: 
   - Input: "A cat sitting on a chair"
   - CLIP Text Encoder → text embeddings [77, 768]

2. **Cross-Attention trong U-Net**:
   ```python
   # Trong mỗi U-Net block
   x = self_attention(x)  # spatial attention
   x = cross_attention(x, text_embeddings)  # text conditioning
   ```

3. **Classifier-Free Guidance**:
   ```python
   # Training: 50% conditional, 50% unconditional
   if random.random() < 0.5:
       condition = text_embedding
   else:
       condition = null_embedding  # học unconditional generation
   
   # Inference: Guidance scale
   ε_pred = ε_uncond + guidance_scale * (ε_cond - ε_uncond)
   ```

### **Training Strategy**:
- **Mixed training**: 50% với text, 50% không có text
- **Null text**: "" (empty string) cho unconditional
- **Text dropout**: Randomly mask text để học robust features

# Mapping từ Paper đến Code Implementation 📁

## VAE Components trong Code

### **Files liên quan**:
- `ldm/models/autoencoder.py`: Main VAE implementation
- `ldm/modules/diffusionmodules/model.py`: Encoder/Decoder architecture
- `configs/autoencoder/`: VAE configurations

### **Key Classes**:
```python
# VAE chính
class AutoencoderKL(nn.Module):
    def __init__(self, ddconfig, embed_dim, ckpt_path=None):
        self.encoder = Encoder(**ddconfig)
        self.decoder = Decoder(**ddconfig) 
        self.quant_conv = nn.Conv2d(ddconfig["z_channels"], embed_dim, 1)
        self.post_quant_conv = nn.Conv2d(embed_dim, ddconfig["z_channels"], 1)
    
    def encode(self, x):
        h = self.encoder(x)
        moments = self.quant_conv(h)
        posterior = DiagonalGaussianDistribution(moments)
        return posterior
    
    def decode(self, z):
        z = self.post_quant_conv(z)
        dec = self.decoder(z)
        return dec
```

---

## U-Net Diffusion Model

### **Files liên quan**:
- `ldm/models/diffusion/ddpm.py`: Main diffusion class
- `ldm/modules/diffusionmodules/openaimodel.py`: U-Net implementation
- `ldm/modules/attention.py`: Attention mechanisms

### **Key Classes**:
```python
# Main Diffusion Model
class LatentDiffusion(DDPM):
    def __init__(self, first_stage_config, cond_stage_config, unet_config, ...):
        # Load pre-trained VAE
        self.instantiate_first_stage(first_stage_config)
        
        # Load conditioning model (CLIP)
        self.instantiate_cond_stage(cond_stage_config) 
        
        # Initialize U-Net
        self.model = DiffusionWrapper(unet_config)
    
    def apply_model(self, x_noisy, t, cond):
        # U-Net forward pass với conditioning
        return self.model(x_noisy, t, cond)
```

### **U-Net Architecture**:
```python
class UNetModel(nn.Module):
    def __init__(self, in_channels, model_channels, out_channels, 
                 attention_resolutions, channel_mult, ...):
        # Time embedding
        self.time_embed = nn.Sequential(...)
        
        # Encoder blocks
        self.input_blocks = nn.ModuleList([...])
        
        # Middle block
        self.middle_block = TimestepEmbedSequential(...)
        
        # Decoder blocks với skip connections
        self.output_blocks = nn.ModuleList([...])
        
        # Cross-attention để inject text conditioning
        self.transformer_blocks = nn.ModuleList([...])
```

---

## Text Conditioning (CLIP)

### **Files liên quan**:
- `ldm/modules/encoders/modules.py`: Text encoders
- `ldm/modules/attention.py`: Cross-attention implementation

### **CLIP Text Encoder**:
```python
class FrozenCLIPEmbedder(nn.Module):
    def __init__(self, version="openai/clip-vit-base-patch32"):
        self.transformer = CLIPTextModel.from_pretrained(version)
        self.transformer.eval()
        
        # Freeze CLIP weights
        for param in self.parameters():
            param.requires_grad = False
    
    def forward(self, text):
        tokens = self.tokenizer(text, truncation=True, max_length=77, 
                               return_tensors="pt", padding="max_length")
        outputs = self.transformer(**tokens)
        return outputs.last_hidden_state
```

### **Cross-Attention Implementation**:
```python
class CrossAttention(nn.Module):
    def forward(self, x, context=None):
        h = x
        q = self.to_q(h)  # query từ spatial features
        
        if context is None:
            context = h  # self-attention
        
        k = self.to_k(context)  # key từ text embeddings
        v = self.to_v(context)  # value từ text embeddings
        
        # Attention computation
        sim = torch.einsum('b i d, b j d -> b i j', q, k) * self.scale
        attn = sim.softmax(dim=-1)
        out = torch.einsum('b i j, b j d -> b i d', attn, v)
        
        return self.to_out(out)
```

# CLIP: Hiểu Sâu về Text-Image Understanding 🔗

## CLIP là gì?

**CLIP** (Contrastive Language-Image Pre-training) là một mô hình AI được OpenAI phát triển năm 2021, có khả năng **hiểu mối liên hệ giữa text và image**.

### 🎯 **Mục tiêu của CLIP**:
- Học được **shared embedding space** cho cả text và image
- Text và image có **same meaning** sẽ có embeddings **gần nhau**
- Text và image **khác meaning** sẽ có embeddings **xa nhau**

### 🧠 **Tại sao CLIP quan trọng?**

Trước CLIP, các AI model thường:
- **Chỉ hiểu text** (GPT, BERT) HOẶC **chỉ hiểu image** (ResNet, EfficientNet)
- **Không thể** kết nối ý nghĩa giữa text và image
- **Cần labeled data** cho mỗi task cụ thể

CLIP có thể:
- **Hiểu cả text và image** cùng một lúc
- **Zero-shot classification**: Phân loại image chỉ bằng text description
- **Semantic similarity**: Tìm image phù hợp với text prompt
- **Flexible**: Không cần training lại cho new tasks

## Kiến trúc của CLIP 🏗️

CLIP gồm **2 encoders chính**:

### 1. **Text Encoder**:
- **Input**: Text string (VD: "A cat sitting on a chair")
- **Tokenization**: Chuyển text thành tokens (words/subwords)
- **Architecture**: Transformer (giống BERT/GPT)
- **Output**: Text embedding vector [512 dim]

### 2. **Image Encoder**: 
- **Input**: Image (VD: ảnh con mèo)
- **Architecture**: Vision Transformer (ViT) hoặc ResNet
- **Output**: Image embedding vector [512 dim]

### 3. **Shared Embedding Space**:
- Cả text và image đều được map vào **cùng một không gian 512-dim**
- **Cosine similarity** được dùng để đo độ tương đồng
- **Contrastive learning** để học embeddings

```
Text: "A cat"     →  [Text Encoder]  →  [0.2, -0.1, 0.8, ...] (512 dims)
Image: 🐱         →  [Image Encoder] →  [0.3, -0.2, 0.7, ...] (512 dims)
                                         ↓
                                   Cosine Similarity = 0.85 (high!)
```

## CLIP được Training như thế nào? 📚

### **Dataset khổng lồ**:
- **400 million** text-image pairs từ internet
- **Diverse**: Mọi chủ đề, ngôn ngữ, style
- **Noisy**: Không cần clean labeling (tự động crawl)

### **Contrastive Learning Process**:

**Ý tưởng**: Trong một batch, mỗi image chỉ match với đúng 1 text của nó.

```python
# Batch example:
Batch = [
    (image1, "A red car"),        # Correct pair
    (image2, "A blue house"),     # Correct pair  
    (image3, "A green tree"),     # Correct pair
    (image4, "A yellow flower")   # Correct pair
]

# CLIP learns:
# image1 should be SIMILAR to "A red car"
# image1 should be DIFFERENT from "A blue house", "A green tree", "A yellow flower"
```

### **Loss Function**:

```python
# Simplified CLIP loss
def clip_loss(image_embeddings, text_embeddings):
    # Compute similarity matrix
    logits = image_embeddings @ text_embeddings.T  # [batch_size, batch_size]
    
    # Diagonal elements should be high (correct pairs)
    # Off-diagonal should be low (incorrect pairs)
    
    # Cross-entropy loss on both directions
    labels = torch.arange(batch_size)  # [0, 1, 2, 3, ...]
    
    loss_i2t = cross_entropy(logits, labels)      # Image to Text
    loss_t2i = cross_entropy(logits.T, labels)    # Text to Image
    
    return (loss_i2t + loss_t2i) / 2
```

In [ ]:
# CLIP Capabilities Demo 🎭

import torch
import torch.nn.functional as F

# Giả lập CLIP embeddings (thực tế sẽ dùng transformers library)
print("🔍 CLIP CAPABILITIES DEMONSTRATION")
print("=" * 50)

# 1. Zero-shot Image Classification
print("\n1️⃣ ZERO-SHOT CLASSIFICATION:")
print("Có thể classify image mà không cần training!")

# Giả sử có 1 image embedding
image_embedding = torch.tensor([0.2, -0.1, 0.8, 0.3])  # 4D for demo

# Các class descriptions
class_texts = [
    "A photo of a cat",
    "A photo of a dog", 
    "A photo of a car",
    "A photo of a tree"
]

# Giả lập text embeddings
text_embeddings = torch.tensor([
    [0.3, -0.2, 0.7, 0.4],  # cat
    [0.1, 0.5, -0.3, 0.2],  # dog
    [-0.4, 0.1, 0.2, -0.1], # car
    [0.6, -0.4, 0.1, 0.8]   # tree
])

# Compute similarities
similarities = F.cosine_similarity(image_embedding.unsqueeze(0), text_embeddings)
print(f"Image similarities với classes:")
for i, (text, sim) in enumerate(zip(class_texts, similarities)):
    print(f"   {text:20s}: {sim:.3f}")

best_match = torch.argmax(similarities)
print(f"\n🎯 Prediction: {class_texts[best_match]} (confidence: {similarities[best_match]:.3f})")

# 2. Text-to-Image Search
print("\n2️⃣ TEXT-TO-IMAGE SEARCH:")
print("Tìm image phù hợp nhất với text query")

# Query text
query = "A cute animal"
query_embedding = torch.tensor([0.25, -0.15, 0.75, 0.35])  # Similar to cat

# Database of images
image_descriptions = [
    "Cat sleeping on sofa",
    "Dog playing in park", 
    "Sports car racing",
    "Mountain landscape"
]

image_embeddings_db = torch.tensor([
    [0.3, -0.2, 0.7, 0.4],   # cat (should match well)
    [0.1, 0.5, -0.3, 0.2],   # dog (should match okay)
    [-0.4, 0.1, 0.2, -0.1],  # car (should not match)
    [0.6, -0.4, 0.1, 0.8]    # landscape (should not match)
])

search_similarities = F.cosine_similarity(query_embedding.unsqueeze(0), image_embeddings_db)
print(f"Query: '{query}'")
print(f"Search results:")

# Sort by similarity
sorted_indices = torch.argsort(search_similarities, descending=True)
for rank, idx in enumerate(sorted_indices, 1):
    print(f"   {rank}. {image_descriptions[idx]:20s}: {search_similarities[idx]:.3f}")

print("\n3️⃣ SEMANTIC UNDERSTANDING:")
print("CLIP hiểu meaning, không chỉ keywords!")

semantics_examples = [
    ("A person riding a bicycle", "Cycling activity", 0.92),
    ("Sunset over ocean", "Beautiful evening seascape", 0.88),
    ("Pizza with pepperoni", "Italian food dish", 0.85),
    ("Code on computer screen", "Programming work", 0.91)
]

print("Examples of semantic similarity:")
for text1, text2, similarity in semantics_examples:
    print(f"   '{text1}' ↔ '{text2}': {similarity}")

print("\n✨ KEY INSIGHTS:")
print("• CLIP không chỉ match keywords, mà hiểu meaning")
print("• Zero-shot learning: không cần training cho new tasks")
print("• Flexible: có thể dùng cho classification, search, generation")
print("• Foundation model cho nhiều multimodal applications")

## CLIP trong Stable Diffusion 🎨

### **Vai trò của CLIP trong Stable Diffusion**:

1. **Text Understanding**: 
   - Input: User prompt "A beautiful sunset over mountains"
   - CLIP Text Encoder: Chuyển thành embedding [77, 768]
   - Output: Rich semantic representation của text

2. **Conditioning Signal**:
   - CLIP embeddings được inject vào U-Net qua **Cross-Attention**
   - Mỗi spatial location trong U-Net có thể "attend" to relevant parts của text
   - Điều này giúp U-Net biết **tạo gì** và **tạo ở đâu**

3. **Why CLIP specifically?**:
   - **Pre-trained**: Đã học từ 400M image-text pairs
   - **Rich representations**: Hiểu complex semantic concepts
   - **Frozen**: Không cần training lại (save compute)
   - **Proven**: Đã được validate trên nhiều tasks

### **Architecture Integration**:

```
User Prompt: "A cat wearing a wizard hat"
       ↓
[CLIP Text Encoder] → Text Embeddings [77, 768]
       ↓
[Cross-Attention trong U-Net]
       ↓  
Spatial Features + Text Features → Enhanced Features
       ↓
Generated Image: 🐱🧙‍♂️
```

### **Tại sao không dùng text encoder khác?**

| Model | Pros | Cons | Use in SD?
|-------|------|------|----------|
| **CLIP** | • Multimodal<br>• Rich semantics<br>• Proven quality | • Limited context (77 tokens) | ✅ SD 1.x |
| **T5** | • Longer context<br>• Pure text model | • Larger size<br>• No image understanding | ✅ SD 2.x |
| **BERT** | • Good text understanding | • No image connection<br>• Less suitable | ❌ |
| **GPT** | • Creative text | • Autoregressive<br>• Overkill | ❌ |

### **CLIP vs T5 trong Stable Diffusion**:

**CLIP** (SD 1.x):
- Compact: 123M parameters
- Fast inference
- Good image-text alignment
- Limited to 77 tokens

**T5** (SD 2.x):
- Larger: 220M - 11B parameters  
- Better long text understanding
- Slower inference
- Can handle complex prompts

### **Practical Impact**:

```python
# CLIP giúp Stable Diffusion hiểu:
"A majestic lion"           → Generates powerful, regal lion
"A cute kitten"             → Generates small, adorable cat
"Lion in cartoon style"     → Understands both subject + style
"Photorealistic lion"       → Understands realism requirement
```

**Without CLIP**: Stable Diffusion sẽ không thể hiểu text prompts!

In [ ]:
# Practical CLIP Implementation for Stable Diffusion 💻

print("🔧 CLIP IMPLEMENTATION IN STABLE DIFFUSION")
print("=" * 55)

# Simulated CLIP Text Encoder (based on real implementation)
class CLIPTextEncoder:
    def __init__(self):
        self.vocab_size = 49408
        self.max_length = 77  # CLIP's context length
        self.embed_dim = 768  # Text embedding dimension
        print(f"📝 CLIP Text Encoder initialized:")
        print(f"   • Vocabulary size: {self.vocab_size:,}")
        print(f"   • Max sequence length: {self.max_length}")
        print(f"   • Embedding dimension: {self.embed_dim}")
    
    def tokenize(self, text):
        """Simulate tokenization process"""
        # Real implementation uses BPE tokenizer
        words = text.lower().split()
        tokens = [49406]  # <start_of_text> token
        
        for word in words[:75]:  # Leave space for start/end tokens
            # Simulate token IDs (real implementation uses BPE)
            token_id = hash(word) % (self.vocab_size - 2) + 1
            tokens.append(token_id)
        
        tokens.append(49407)  # <end_of_text> token
        
        # Pad to max_length
        while len(tokens) < self.max_length:
            tokens.append(0)  # <pad> token
            
        return tokens[:self.max_length]
    
    def encode(self, text):
        """Convert text to embeddings"""
        tokens = self.tokenize(text)
        print(f"\n🔤 Text processing:")
        print(f"   Input: '{text}'")
        print(f"   Tokens: {len([t for t in tokens if t != 0])} real tokens")
        print(f"   Padded to: {len(tokens)} tokens")
        
        # Simulate embeddings (real implementation uses transformer)
        import torch
        embeddings = torch.randn(self.max_length, self.embed_dim)
        
        print(f"   Output shape: {list(embeddings.shape)}")
        return embeddings

# Demo CLIP usage
clip_encoder = CLIPTextEncoder()

# Test various prompts
test_prompts = [
    "A beautiful sunset over mountains",
    "A cat wearing a wizard hat in a magical forest", 
    "Photorealistic portrait of a woman with blue eyes",
    "Abstract painting in the style of Van Gogh"
]

print("\n🎨 PROCESSING VARIOUS PROMPTS:")
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n--- Example {i} ---")
    embeddings = clip_encoder.encode(prompt)
    
    # Simulate using embeddings in U-Net
    print(f"   ✅ Ready for Cross-Attention in U-Net")
    print(f"   ✅ Will guide image generation process")

print("\n🧠 HOW CLIP EMBEDDINGS GUIDE GENERATION:")
print("""
1. **Rich Semantics**: 
   - "beautiful" → aesthetic qualities
   - "sunset" → lighting, colors, time of day
   - "mountains" → landscape, composition

2. **Style Understanding**:
   - "photorealistic" → detailed, camera-like
   - "abstract" → non-representational
   - "Van Gogh style" → brushstrokes, colors

3. **Compositional Hints**:
   - "portrait" → close-up, centered
   - "landscape" → wide view, horizon
   - "in a forest" → background elements
""")

print("\n🎯 KEY TECHNICAL DETAILS:")
print("• CLIP embeddings shape: [77, 768]")
print("• Each token gets 768-dimensional representation")
print("• Cross-attention uses these as Keys & Values")
print("• Spatial features from U-Net become Queries")
print("• This allows each pixel to 'look at' relevant text parts")

print("\n✨ CLIP makes text-to-image generation possible!")
print("Without CLIP, Stable Diffusion would be just noise → noise 🌪️")
print("With CLIP, it becomes meaningful: text → beautiful images 🎨")

In [ ]:
# ROADMAP: Dựng lại Stable Diffusion từ đầu 🛠️

print("=== BƯỚC 1: CHUẨN BỊ DATASET VÀ INFRASTRUCTURE ===")
print("""
1.1. Dataset Preparation:
   • Text-Image pairs: LAION-400M, CC12M, hoặc custom dataset
   • Image preprocessing: Resize to 512x512, normalize [-1, 1]
   • Text preprocessing: Tokenization, max length 77

1.2. Infrastructure:
   • Multi-GPU setup (8x A100 recommended)
   • Distributed training framework (PyTorch Lightning)
   • Wandb/TensorBoard cho monitoring
   • Large storage for datasets (TB scale)
""")

print("\n=== BƯỚC 2: IMPLEMENT VAE (First Stage Model) ===")
print("""
2.1. VAE Architecture:
   • Encoder: ResNet-based với downsampling blocks
   • Decoder: Symmetric upsampling blocks
   • Latent space: 4 channels, 64x64 (cho 512x512 input)
   • KL regularization

2.2. Training VAE:
   • Loss: Reconstruction + β*KL + λ*Perceptual + Adversarial
   • Perceptual loss: VGG16 features
   • Discriminator: PatchGAN for adversarial loss
   • Training time: ~1 tuần với 8 GPUs

2.3. VAE Validation:
   • Reconstruction quality: LPIPS, SSIM, FID
   • Compression efficiency: File size reduction
   • Latent space interpolation
""")

print("\n=== BƯỚC 3: IMPLEMENT U-NET DIFFUSION MODEL ===") 
print("""
3.1. U-Net Architecture:
   • Input: 4-channel latent + time embedding
   • Encoder-Decoder với skip connections
   • Multi-scale attention layers
   • Group normalization
   • SiLU activation

3.2. Diffusion Components:
   • Noise scheduler: Linear or cosine β schedule
   • Timestep embedding: Sinusoidal positional encoding
   • Loss function: Simple MSE loss
   • Sampling: DDPM or DDIM

3.3. Training Process:
   • Encode images với pre-trained VAE
   • Random timestep sampling
   • Noise prediction training
   • Training time: ~2-3 tuần với 8 GPUs
""")

print("\n=== BƯỚC 4: ADD TEXT CONDITIONING ===")
print("""
4.1. Text Encoder:
   • CLIP Text Encoder (frozen)
   • Tokenization: max 77 tokens
   • Output: [batch, 77, 768] embeddings

4.2. Cross-Attention:
   • Modify U-Net blocks
   • Query: spatial features, Key/Value: text embeddings
   • Multi-head attention

4.3. Classifier-Free Guidance:
   • 50% conditional, 50% unconditional training
   • Null text embedding cho unconditional
   • Guidance scale trong inference

4.4. Training Strategy:
   • Mixed conditioning training
   • Text dropout techniques
   • Training time: ~1-2 tuần additional
""")

print("\n=== BƯỚC 5: OPTIMIZATION VÀ INFERENCE ===")
print("""
5.1. Training Optimizations:
   • Mixed precision training (FP16)
   • Gradient checkpointing
   • EMA (Exponential Moving Average) weights
   • Learning rate scheduling

5.2. Inference Optimizations:
   • DDIM sampling (fewer steps)
   • xFormers attention (memory efficient)
   • Model quantization
   • TensorRT optimization

5.3. Evaluation Metrics:
   • FID (Fréchet Inception Distance)
   • CLIP Score cho text alignment
   • Human evaluation
   • Aesthetic quality scores
""")

print("\n=== BƯỚC 6: DEPLOYMENT VÀ SCALING ===")
print("""
6.1. Model Serving:
   • API wrapper (FastAPI/Flask)
   • Batch inference
   • Queue management
   • Load balancing

6.2. User Interface:
   • Web interface (Gradio/Streamlit)
   • Image generation controls
   • Prompt engineering tools
   • Gallery và sharing features

6.3. Advanced Features:
   • Image-to-image generation
   • Inpainting capability
   • ControlNet integration
   • LoRA fine-tuning support
""")

# Estimated Timeline
print("\n🕐 TIMELINE ESTIMATE:")
print("VAE Training: 1-2 weeks")
print("U-Net Training: 2-3 weeks") 
print("Text Conditioning: 1-2 weeks")
print("Optimization & Testing: 1 week")
print("TOTAL: 5-8 weeks với 8x A100 GPUs")

print("\n💰 COST ESTIMATE:")
print("8x A100 cloud cost: ~$20-30/hour")
print("Total training cost: $50,000 - $100,000 USD")
print("Alternative: Start với smaller model, scale up gradually")

In [2]:
# PRACTICAL IMPLEMENTATION: Code Structure 💻

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pytorch_lightning as pl

# =============================================================================
# BƯỚC 1: VAE Implementation
# =============================================================================

class VAEEncoder(nn.Module):
    """VAE Encoder: Image → Latent"""
    def __init__(self, in_channels=3, latent_channels=4, ch_mult=[1,2,4,8]):
        super().__init__()
        self.conv_in = nn.Conv2d(in_channels, 128, 3, padding=1)
        
        # Downsampling blocks
        self.down_blocks = nn.ModuleList()
        ch = 128
        for mult in ch_mult:
            self.down_blocks.append(nn.Sequential(
                nn.Conv2d(ch, ch*mult, 4, stride=2, padding=1),
                nn.GroupNorm(32, ch*mult),
                nn.SiLU()
            ))
            ch = ch * mult
        
        # Output projection
        self.norm_out = nn.GroupNorm(32, ch)
        self.conv_out = nn.Conv2d(ch, latent_channels*2, 3, padding=1)  # mu + logvar
    
    def forward(self, x):
        h = self.conv_in(x)
        for block in self.down_blocks:
            h = block(h)
        
        h = self.norm_out(h)
        h = F.silu(h)
        moments = self.conv_out(h)
        
        # Split into mu and logvar
        mu, logvar = moments.chunk(2, dim=1)
        return mu, logvar

class VAEDecoder(nn.Module):
    """VAE Decoder: Latent → Image"""
    def __init__(self, latent_channels=4, out_channels=3, ch_mult=[8,4,2,1]):
        super().__init__()
        ch = 128 * ch_mult[0]
        self.conv_in = nn.Conv2d(latent_channels, ch, 3, padding=1)
        
        # Upsampling blocks
        self.up_blocks = nn.ModuleList()
        for mult in ch_mult:
            self.up_blocks.append(nn.Sequential(
                nn.ConvTranspose2d(ch, 128*mult, 4, stride=2, padding=1),
                nn.GroupNorm(32, 128*mult),
                nn.SiLU()
            ))
            ch = 128 * mult
        
        # Output projection
        self.norm_out = nn.GroupNorm(32, ch)
        self.conv_out = nn.Conv2d(ch, out_channels, 3, padding=1)
    
    def forward(self, z):
        h = self.conv_in(z)
        for block in self.up_blocks:
            h = block(h)
        
        h = self.norm_out(h)
        h = F.silu(h)
        return torch.tanh(self.conv_out(h))  # Output in [-1, 1]

class VAE(pl.LightningModule):
    """Complete VAE Model"""
    def __init__(self, lr=1e-4, beta=1.0, perceptual_weight=1.0):
        super().__init__()
        self.encoder = VAEEncoder()
        self.decoder = VAEDecoder()
        self.lr = lr
        self.beta = beta
        self.perceptual_weight = perceptual_weight
        
        # Perceptual loss (VGG)
        from torchvision.models import vgg16
        vgg = vgg16(pretrained=True).features[:16]  # Up to relu3_3
        for param in vgg.parameters():
            param.requires_grad = False
        self.perceptual_net = vgg
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        recon = self.decoder(z)
        return recon, mu, logvar
    
    def training_step(self, batch, batch_idx):
        x, _ = batch  # Ignore labels for now
        recon, mu, logvar = self(x)
        
        # Reconstruction loss
        recon_loss = F.mse_loss(recon, x)
        
        # KL divergence
        kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
        
        # Perceptual loss
        x_feat = self.perceptual_net(x)
        recon_feat = self.perceptual_net(recon)
        perceptual_loss = F.mse_loss(recon_feat, x_feat)
        
        # Total loss
        loss = recon_loss + self.beta * kl_loss + self.perceptual_weight * perceptual_loss
        
        self.log_dict({
            'train_loss': loss,
            'recon_loss': recon_loss,
            'kl_loss': kl_loss,
            'perceptual_loss': perceptual_loss
        })
        
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

# =============================================================================
# BƯỚC 2: U-Net Diffusion Model
# =============================================================================

class TimeEmbedding(nn.Module):
    """Sinusoidal time embedding"""
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
        
    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = torch.log(torch.tensor(10000.0)) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings

class UNetBlock(nn.Module):
    """Basic U-Net residual block với time embedding"""
    def __init__(self, in_ch, out_ch, time_emb_dim, dropout=0.1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
        self.time_mlp = nn.Linear(time_emb_dim, out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.norm1 = nn.GroupNorm(8, out_ch)
        self.norm2 = nn.GroupNorm(8, out_ch)
        self.dropout = nn.Dropout(dropout)
        
        if in_ch != out_ch:
            self.shortcut = nn.Conv2d(in_ch, out_ch, 1)
        else:
            self.shortcut = nn.Identity()
    
    def forward(self, x, time_emb):
        h = self.conv1(x)
        h = self.norm1(h)
        h += self.time_mlp(time_emb)[:, :, None, None]
        h = F.silu(h)
        h = self.dropout(h)
        
        h = self.conv2(h)
        h = self.norm2(h)
        h = F.silu(h)
        
        return h + self.shortcut(x)

class SimpleUNet(nn.Module):
    """Simplified U-Net cho Diffusion"""
    def __init__(self, in_channels=4, out_channels=4, features=[64, 128, 256, 512]):
        super().__init__()
        
        # Time embedding
        time_emb_dim = features[0] * 4
        self.time_embedding = TimeEmbedding(time_emb_dim)
        self.time_mlp = nn.Sequential(
            nn.Linear(time_emb_dim, time_emb_dim),
            nn.SiLU(),
            nn.Linear(time_emb_dim, time_emb_dim)
        )
        
        # Encoder
        self.encoder = nn.ModuleList()
        prev_ch = in_channels
        for feat in features:
            self.encoder.append(UNetBlock(prev_ch, feat, time_emb_dim))
            prev_ch = feat
        
        # Middle
        self.middle = UNetBlock(features[-1], features[-1], time_emb_dim)
        
        # Decoder
        self.decoder = nn.ModuleList()
        for feat in reversed(features[:-1]):
            self.decoder.append(UNetBlock(prev_ch + feat, feat, time_emb_dim))
            prev_ch = feat
        
        # Output
        self.output = nn.Conv2d(features[0], out_channels, 1)
    
    def forward(self, x, timesteps):
        # Time embedding
        t_emb = self.time_embedding(timesteps)
        t_emb = self.time_mlp(t_emb)
        
        # Encoder
        skip_connections = []
        for encoder_block in self.encoder:
            x = encoder_block(x, t_emb)
            skip_connections.append(x)
            x = F.max_pool2d(x, 2)
        
        # Middle
        x = self.middle(x, t_emb)
        
        # Decoder
        for decoder_block, skip in zip(self.decoder, reversed(skip_connections[:-1])):
            x = F.interpolate(x, scale_factor=2, mode='nearest')
            x = torch.cat([x, skip], dim=1)
            x = decoder_block(x, t_emb)
        
        return self.output(x)

print("✅ VAE và U-Net implementation ready!")
print("Next: Text conditioning với CLIP và Cross-attention")
print("Tổng cộng: ~500 lines code cho base implementation")

/home/loc-dang/Projects/reai_lab/stable_diffusion/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ VAE và U-Net implementation ready!
Next: Text conditioning với CLIP và Cross-attention
Tổng cộng: ~500 lines code cho base implementation


# Tổng kết: Roadmap dựng lại Stable Diffusion 🎯

## 📋 Checklist hoàn chỉnh

### ✅ **Đã hiểu**:
- [x] Architecture tổng thể (VAE + U-Net + CLIP)
- [x] 3 giai đoạn training
- [x] Loss functions cho từng component
- [x] Mapping từ paper đến code
- [x] Implementation skeleton

### 🔄 **Cần implement**:
- [ ] **VAE**: Encoder + Decoder + Training loop
- [ ] **U-Net**: Diffusion model với time embedding
- [ ] **Text Conditioning**: CLIP + Cross-attention
- [ ] **Training Pipeline**: DataLoader + Optimization
- [ ] **Inference**: Sampling algorithms (DDPM/DDIM)

## 🎯 **Next Steps**

### **Lựa chọn 1: Start Small** (Recommended)
```python
# Proof of concept với smaller model
image_size = 128  # instead of 512
latent_size = 16  # instead of 64
training_steps = 100K  # instead of millions
dataset = "CIFAR-10"  # instead of LAION-400M
```

### **Lựa chọn 2: Full Scale**
```python
# Production-ready implementation
image_size = 512
latent_size = 64
training_steps = 1M+
dataset = "LAION-400M"
hardware = "8x A100 GPUs"
```

### **Lựa chọn 3: Fine-tuning Approach**
```python
# Start from pre-trained weights
base_model = "runwayml/stable-diffusion-v1-5"
task = "Fine-tune trên custom dataset"
compute = "Single A100"
time = "1-2 tuần"
```

## 🛠️ **Tools và Resources cần có**

### **Development**:
- PyTorch 2.0+
- PyTorch Lightning
- Transformers (Hugging Face)
- xFormers (memory optimization)
- Wandb (experiment tracking)

### **Data**:
- LAION-400M (nếu full scale)
- CC12M (smaller alternative)
- Custom dataset (nếu specialized use case)

### **Compute**:
- **Minimum**: 1x RTX 4090 (24GB VRAM)
- **Recommended**: 4-8x A100 (40-80GB VRAM)
- **Storage**: 10-100TB for datasets

## 💡 **Key Insights từ Analysis**

1. **Stable Diffusion ≠ 1 model**
   - Là hệ thống gồm 3 components
   - Mỗi component train riêng biệt
   - Kết hợp lại thành pipeline hoàn chỉnh

2. **VAE là foundation**
   - Quality của VAE quyết định quality cuối cùng
   - Perceptual loss rất quan trọng
   - Compression ratio impact performance

3. **Diffusion trong latent space**
   - 64x faster than pixel space
   - Vẫn maintain high quality
   - Enable high-resolution generation

4. **Text conditioning là key differentiator**
   - CLIP text encoder
   - Cross-attention mechanism
   - Classifier-free guidance for control

## 🚀 **Recommendation**

Bắt đầu với **Lựa chọn 1** (Start Small) để:
- Hiểu sâu implementation details
- Test và debug code
- Validate approach
- Sau đó scale up dần dần

**Timeline thực tế**:
- Week 1-2: VAE implementation và training
- Week 3-4: U-Net diffusion model
- Week 5-6: Text conditioning
- Week 7-8: Integration và optimization

**Ready để bắt đầu implement! 🎉**

In [3]:
# 🎊 FINAL CELEBRATION & SUMMARY

print("🎯" * 20)
print("     STABLE DIFFUSION MASTERY ACHIEVED!")
print("🎯" * 20)

# What we learned
learned_concepts = [
    "Perceptual Loss với VGG features",
    "VAE Encoder/Decoder architecture", 
    "Diffusion forward/reverse process",
    "U-Net với time embeddings",
    "CLIP text encoding",
    "Cross-attention mechanism",
    "Classifier-free guidance",
    "3-phase training pipeline",
    "Latent space compression",
    "DDPM/DDIM sampling"
]

print("\n📚 CONCEPTS MASTERED:")
for i, concept in enumerate(learned_concepts, 1):
    print(f"   {i:2d}. ✅ {concept}")

# Implementation progress
code_components = {
    "VAE Encoder": "✅ Complete",
    "VAE Decoder": "✅ Complete", 
    "Training Loop": "✅ Complete",
    "U-Net Architecture": "✅ Complete",
    "Time Embedding": "✅ Complete",
    "Text Conditioning": "🔄 Skeleton ready",
    "Cross-Attention": "🔄 Skeleton ready",
    "Sampling Pipeline": "🔄 Next step",
    "Full Integration": "🔄 Next step"
}

print("\n💻 CODE IMPLEMENTATION STATUS:")
for component, status in code_components.items():
    print(f"   {component:20s} : {status}")

print("\n🎯 IMMEDIATE NEXT STEPS:")
print("   1. 🔄 Complete text conditioning implementation")
print("   2. 🔄 Build full training pipeline")
print("   3. 🔄 Test với mini dataset (CIFAR-10)")
print("   4. 🔄 Scale up to real datasets")
print("   5. 🔄 Deploy và share với community")

print("\n🏆 ACHIEVEMENT UNLOCKED:")
print("   🥇 Stable Diffusion Architecture Expert")
print("   🥈 Diffusion Models Implementation Specialist") 
print("   🥉 AI Art Generation System Builder")

print("\n" + "🎨" * 25)
print("  FROM ZERO TO DIFFUSION HERO!")
print("🎨" * 25)

print("\n🚀 Ready to change the world với AI creativity!")
print("💪 Knowledge is power - use it wisely!")
print("🌟 The future of AI art starts with YOU!")

🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
     STABLE DIFFUSION MASTERY ACHIEVED!
🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯

📚 CONCEPTS MASTERED:
    1. ✅ Perceptual Loss với VGG features
    2. ✅ VAE Encoder/Decoder architecture
    3. ✅ Diffusion forward/reverse process
    4. ✅ U-Net với time embeddings
    5. ✅ CLIP text encoding
    6. ✅ Cross-attention mechanism
    7. ✅ Classifier-free guidance
    8. ✅ 3-phase training pipeline
    9. ✅ Latent space compression
   10. ✅ DDPM/DDIM sampling

💻 CODE IMPLEMENTATION STATUS:
   VAE Encoder          : ✅ Complete
   VAE Decoder          : ✅ Complete
   Training Loop        : ✅ Complete
   U-Net Architecture   : ✅ Complete
   Time Embedding       : ✅ Complete
   Text Conditioning    : 🔄 Skeleton ready
   Cross-Attention      : 🔄 Skeleton ready
   Sampling Pipeline    : 🔄 Next step
   Full Integration     : 🔄 Next step

🎯 IMMEDIATE NEXT STEPS:
   1. 🔄 Complete text conditioning implementation
   2. 🔄 Build full training pipeline
   3. 🔄 Test với mini dataset (CIFAR-10)
   4. 🔄 S

In [4]:
# 📚 ROADMAP CHO PAPER: High-Resolution Image Synthesis with Latent Diffusion Models

print("🎯 ROADMAP ĐỌC HIỂU LATENT DIFFUSION MODELS PAPER")
print("=" * 60)

print("📄 Paper target: 'High-Resolution Image Synthesis with Latent Diffusion Models'")
print("🔗 ArXiv: 2112.10752v2")
print("📅 Submitted: Dec 2021")
print("👥 Authors: Robin Rombach, Andreas Blattmann, et al.")
print("🏢 Institution: LMU Munich, IWR Heidelberg")
print("💡 Nickname: 'Stable Diffusion Paper'")

print("\n" + "="*60)
print("🚨 CRITICAL FOUNDATION PAPERS - ĐỌC TRƯỚC TIÊN")
print("="*60)

critical_papers = [
    {
        "priority": "🔥 MUST READ #1",
        "title": "Denoising Diffusion Probabilistic Models",
        "authors": "Jonathan Ho, Ajay Jain, Pieter Abbeel",
        "arxiv": "2006.11239",
        "year": "2020",
        "venue": "NeurIPS 2020",
        "why_critical": [
            "🎯 Định nghĩa core concept của diffusion models",
            "🎯 Forward process q(x₁:T|x₀) và reverse process pθ(x₀:T₋₁|xT)",
            "🎯 Variational lower bound derivation",
            "🎯 Simplified loss function: ||ε - εθ(xt,t)||²",
            "🎯 DDPM sampling algorithm"
        ],
        "key_sections": [
            "Section 2: Background",
            "Section 3: Diffusion models",
            "Section 4: Experiments",
            "Algorithm 1: Training",
            "Algorithm 2: Sampling"
        ],
        "time_needed": "4-6 hours",
        "difficulty": "⭐⭐⭐⭐",
        "concepts_needed": [
            "Markov chains",
            "Variational inference basics",
            "Gaussian distributions",
            "Neural networks"
        ]
    },
    
    {
        "priority": "🔥 MUST READ #2", 
        "title": "Auto-Encoding Variational Bayes",
        "authors": "Diederik P. Kingma, Max Welling",
        "arxiv": "1312.6114",
        "year": "2013",
        "venue": "ICLR 2014",
        "why_critical": [
            "🎯 VAE framework - foundation cho latent space work",
            "🎯 Encoder-decoder architecture",
            "🎯 Reparameterization trick",
            "🎯 KL divergence regularization",
            "🎯 Evidence Lower Bound (ELBO)"
        ],
        "key_sections": [
            "Section 2.1: Problem scenario",
            "Section 2.2: The variational bound", 
            "Section 2.3: The reparameterization trick",
            "Section 2.4: Estimator"
        ],
        "time_needed": "3-4 hours",
        "difficulty": "⭐⭐⭐",
        "concepts_needed": [
            "Bayesian inference",
            "Variational methods",
            "Information theory basics"
        ]
    },
    
    {
        "priority": "🔥 MUST READ #3",
        "title": "Attention Is All You Need", 
        "authors": "Vaswani, Shazeer, Parmar, et al.",
        "arxiv": "1706.03762",
        "year": "2017",
        "venue": "NeurIPS 2017",
        "why_critical": [
            "🎯 Self-attention mechanism",
            "🎯 Multi-head attention",
            "🎯 Cross-attention (key cho text conditioning)",
            "🎯 Positional encoding",
            "🎯 Transformer blocks"
        ],
        "key_sections": [
            "Section 3.1: Encoder and Decoder Stacks",
            "Section 3.2: Attention", 
            "Section 3.2.1: Scaled Dot-Product Attention",
            "Section 3.2.2: Multi-Head Attention"
        ],
        "time_needed": "3-4 hours",
        "difficulty": "⭐⭐⭐",
        "concepts_needed": [
            "Linear algebra",
            "Neural networks",
            "Sequence modeling"
        ]
    }
]

for paper in critical_papers:
    print(f"\n{paper['priority']}")
    print(f"📖 Title: {paper['title']}")
    print(f"👥 Authors: {paper['authors']}")
    print(f"🔗 ArXiv: {paper['arxiv']}")
    print(f"📅 Year: {paper['year']} ({paper['venue']})")
    print(f"⏱️ Time needed: {paper['time_needed']}")
    print(f"🌟 Difficulty: {paper['difficulty']}")
    
    print(f"\n💡 Why critical:")
    for reason in paper['why_critical']:
        print(f"   {reason}")
    
    print(f"\n📚 Key sections to focus on:")
    for section in paper['key_sections']:
        print(f"   • {section}")
    
    print(f"\n🧠 Prerequisites:")
    for concept in paper['concepts_needed']:
        print(f"   • {concept}")

print("\n" + "="*60)
print("⚡ IMPORTANT SUPPORTING PAPERS")
print("="*60)

supporting_papers = [
    {
        "title": "Learning Transferable Visual Models From Natural Language Supervision",
        "nickname": "CLIP",
        "authors": "Radford et al. (OpenAI)",
        "arxiv": "2103.00020",
        "year": "2021",
        "why_important": [
            "🔸 Text encoder trong Stable Diffusion",
            "🔸 Contrastive learning framework",
            "🔸 Joint text-image embedding space",
            "🔸 Zero-shot capabilities"
        ],
        "connection": "Used as conditioning mechanism trong LDM"
    },
    
    {
        "title": "Denoising Diffusion Implicit Models",
        "nickname": "DDIM", 
        "authors": "Jiaming Song, Chenlin Meng, Stefano Ermon",
        "arxiv": "2010.02502",
        "year": "2020",
        "why_important": [
            "🔸 Deterministic sampling process",
            "🔸 Faster inference (fewer steps)",
            "🔸 Better speed-quality tradeoff",
            "🔸 Non-Markovian formulation"
        ],
        "connection": "Alternative sampling method mentioned trong LDM"
    },
    
    {
        "title": "Generative Adversarial Networks",
        "nickname": "GAN",
        "authors": "Ian Goodfellow et al.",
        "arxiv": "1406.2661", 
        "year": "2014",
        "why_important": [
            "🔸 Adversarial training concept",
            "🔸 Generator-discriminator framework",
            "🔸 Comparison baseline trong paper",
            "🔸 Understanding of generative models landscape"
        ],
        "connection": "Compared against trong experiments"
    },
    
    {
        "title": "Taming Transformers for High-Resolution Image Synthesis",
        "nickname": "VQGAN",
        "authors": "Patrick Esser et al.",
        "arxiv": "2012.09841",
        "year": "2020", 
        "why_important": [
            "🔸 High-resolution image synthesis",
            "🔸 Vector quantization techniques",
            "🔸 Perceptual losses",
            "🔸 Comparison với autoregressive models"
        ],
        "connection": "Baseline comparison và related work"
    }
]

for paper in supporting_papers:
    print(f"\n📑 {paper['title']} ({paper['nickname']})")
    print(f"👥 {paper['authors']}")
    print(f"🔗 ArXiv: {paper['arxiv']} ({paper['year']})")
    print(f"🔄 Connection: {paper['connection']}")
    print(f"📌 Why important:")
    for reason in paper['why_important']:
        print(f"   {reason}")

print("\n" + "="*60)
print("📅 SUGGESTED 4-WEEK READING SCHEDULE")
print("="*60)

weekly_schedule = [
    {
        "week": "Week 1: Foundation Concepts",
        "papers": [
            "Auto-Encoding Variational Bayes (VAE)",
            "Attention Is All You Need (Transformers)"
        ],
        "goals": [
            "Understand latent space representation",
            "Master attention mechanisms", 
            "Learn encoder-decoder architectures"
        ],
        "time": "6-8 hours",
        "deliverable": "Implement simple VAE và attention từ scratch"
    },
    
    {
        "week": "Week 2: Diffusion Deep Dive",
        "papers": [
            "Denoising Diffusion Probabilistic Models (DDPM)"
        ],
        "goals": [
            "Master forward và reverse diffusion process",
            "Understand variational bound derivation",
            "Learn DDPM training và sampling algorithms"
        ],
        "time": "6-8 hours", 
        "deliverable": "Implement DDPM on toy dataset (MNIST/CIFAR)"
    },
    
    {
        "week": "Week 3: Advanced Topics",
        "papers": [
            "CLIP (text conditioning)",
            "DDIM (fast sampling)",
            "Skim GAN và VQGAN papers"
        ],
        "goals": [
            "Understand text-image joint embeddings",
            "Learn faster sampling techniques",
            "Comparison với other generative models"
        ],
        "time": "5-7 hours",
        "deliverable": "Add text conditioning to diffusion model"
    },
    
    {
        "week": "Week 4: Latent Diffusion Models",
        "papers": [
            "High-Resolution Image Synthesis with Latent Diffusion Models",
            "Re-read key sections từ previous papers"
        ],
        "goals": [
            "🎯 MASTER THE TARGET PAPER",
            "Connect all concepts together",
            "Understand practical implementation details"
        ],
        "time": "8-10 hours",
        "deliverable": "Complete understanding + implementation plan"
    }
]

for week in weekly_schedule:
    print(f"\n📅 {week['week']}")
    print(f"📚 Papers:")
    for paper in week['papers']:
        print(f"   • {paper}")
    
    print(f"🎯 Goals:")
    for goal in week['goals']:
        print(f"   • {goal}")
    
    print(f"⏱️ Time: {week['time']}")
    print(f"📝 Deliverable: {week['deliverable']}")

print("\n" + "="*60)
print("🧩 CONCEPT DEPENDENCY MAP")
print("="*60)

dependency_map = {
    "Latent Diffusion Models": {
        "depends_on": ["VAE", "DDPM", "Transformers"],
        "enables": "High-res image synthesis trong latent space"
    },
    "VAE": {
        "depends_on": ["Variational Inference", "Neural Networks"],
        "enables": "Latent space representation cho images"
    },
    "DDPM": {
        "depends_on": ["Markov Chains", "Variational Bounds"],
        "enables": "Iterative denoising generation process"
    },
    "Transformers": {
        "depends_on": ["Attention Mechanism", "Deep Learning"],
        "enables": "Cross-attention cho text conditioning"
    },
    "CLIP": {
        "depends_on": ["Transformers", "Contrastive Learning"],
        "enables": "Text-image joint understanding"
    }
}

print("🔄 How concepts build on each other:")
for concept, info in dependency_map.items():
    print(f"\n{concept}:")
    print(f"   Depends on: {', '.join(info['depends_on'])}")
    print(f"   Enables: {info['enables']}")

print("\n" + "="*60)
print("🎯 SUCCESS CHECKLIST")
print("="*60)

success_checklist = [
    "✅ Understand forward diffusion: x₀ → xT (adding noise)",
    "✅ Understand reverse diffusion: xT → x₀ (denoising)",
    "✅ Know why work trong latent space instead of pixel space",
    "✅ Understand VAE encoder: x → z và decoder: z → x", 
    "✅ Know how cross-attention injects text conditioning",
    "✅ Understand the simplified loss: ||ε - εθ(zt,t,c)||²",
    "✅ Can explain classifier-free guidance",
    "✅ Know differences giữa DDPM và DDIM sampling",
    "✅ Understand computational advantages của LDM",
    "✅ Can implement basic components từ scratch"
]

print("After completing this roadmap, bạn should:")
for item in success_checklist:
    print(f"   {item}")

print("\n" + "="*60)
print("💡 READING STRATEGIES")
print("="*60)

reading_tips = [
    "📖 First pass: Skim để get big picture",
    "📝 Second pass: Deep read với note-taking",
    "🔢 Focus on key equations và their intuitions",
    "🖼️ Draw diagrams cho architectures và data flows",
    "💻 Implement toy versions để test understanding",
    "🤔 Ask yourself: 'Why did they make this choice?'",
    "🔗 Connect concepts across papers",
    "⏸️ Take breaks khi encounter difficult sections",
    "👥 Discuss với others hoặc online communities",
    "🔄 Revisit difficult concepts multiple times"
]

for tip in reading_tips:
    print(f"   {tip}")

print("\n🚀 START WITH VAE PAPER - IT'S THE MOST ACCESSIBLE!")
print("Then move to Transformers, followed by DDPM.")
print("Good luck on your journey to understanding Latent Diffusion Models! 🎯✨")

🎯 ROADMAP ĐỌC HIỂU LATENT DIFFUSION MODELS PAPER
📄 Paper target: 'High-Resolution Image Synthesis with Latent Diffusion Models'
🔗 ArXiv: 2112.10752v2
📅 Submitted: Dec 2021
👥 Authors: Robin Rombach, Andreas Blattmann, et al.
🏢 Institution: LMU Munich, IWR Heidelberg
💡 Nickname: 'Stable Diffusion Paper'

🚨 CRITICAL FOUNDATION PAPERS - ĐỌC TRƯỚC TIÊN

🔥 MUST READ #1
📖 Title: Denoising Diffusion Probabilistic Models
👥 Authors: Jonathan Ho, Ajay Jain, Pieter Abbeel
🔗 ArXiv: 2006.11239
📅 Year: 2020 (NeurIPS 2020)
⏱️ Time needed: 4-6 hours
🌟 Difficulty: ⭐⭐⭐⭐

💡 Why critical:
   🎯 Định nghĩa core concept của diffusion models
   🎯 Forward process q(x₁:T|x₀) và reverse process pθ(x₀:T₋₁|xT)
   🎯 Variational lower bound derivation
   🎯 Simplified loss function: ||ε - εθ(xt,t)||²
   🎯 DDPM sampling algorithm

📚 Key sections to focus on:
   • Section 2: Background
   • Section 3: Diffusion models
   • Section 4: Experiments
   • Algorithm 1: Training
   • Algorithm 2: Sampling

🧠 Prerequisites:
   